In [1]:
import pandas as pd
import numpy as np


In [8]:
df = pd.read_csv("../data/raw/bank_customer_churn.csv")

In [9]:
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [10]:
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"])


In [11]:
df.columns = (
    df.columns
    .str.lower()
    .str.strip()
)


In [12]:
df.isnull().sum()


creditscore        0
geography          0
gender             0
age                0
tenure             0
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
exited             0
dtype: int64

In [13]:
df["churn_status"] = df["exited"].map({0: "Retained", 1: "Churned"})
df["activity_status"] = df["isactivemember"].map({0: "Inactive", 1: "Active"})
df["credit_card_status"] = df["hascrcard"].map({0: "No Card", 1: "Has Card"})


In [14]:
age_bins = [18, 25, 35, 45, 55, 65, 100]
age_labels = ["18–25", "26–35", "36–45", "46–55", "56–65", "65+"]

df["age_group"] = pd.cut(
    df["age"],
    bins=age_bins,
    labels=age_labels
)


In [15]:
balance_bins = [-1, 0, 50000, 100000, 200000, df["balance"].max()]
balance_labels = [
    "Zero Balance",
    "Low Balance",
    "Mid Balance",
    "High Balance",
    "Very High Balance"
]

df["balance_segment"] = pd.cut(
    df["balance"],
    bins=balance_bins,
    labels=balance_labels
)


In [16]:
df["geography"] = df["geography"].str.title()
df["gender"] = df["gender"].str.title()


In [17]:
# Zero balance but active customers
df[(df["balance"] == 0) & (df["isactivemember"] == 1)].shape


(1873, 16)

In [18]:
# Churn rate sanity check
df["churn_status"].value_counts(normalize=True) * 100


churn_status
Retained    79.63
Churned     20.37
Name: proportion, dtype: float64

In [19]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   creditscore         10000 non-null  int64   
 1   geography           10000 non-null  object  
 2   gender              10000 non-null  object  
 3   age                 10000 non-null  int64   
 4   tenure              10000 non-null  int64   
 5   balance             10000 non-null  float64 
 6   numofproducts       10000 non-null  int64   
 7   hascrcard           10000 non-null  int64   
 8   isactivemember      10000 non-null  int64   
 9   estimatedsalary     10000 non-null  float64 
 10  exited              10000 non-null  int64   
 11  churn_status        10000 non-null  object  
 12  activity_status     10000 non-null  object  
 13  credit_card_status  10000 non-null  object  
 14  age_group           9978 non-null   category
 15  balance_segment     10000 non-null  c

In [20]:
df.head()


,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited,churn_status,activity_status,credit_card_status,age_group,balance_segment
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,Churned,Active,Has Card,36–45,Zero Balance
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,Retained,Active,No Card,36–45,Mid Balance
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,Churned,Inactive,Has Card,36–45,High Balance
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,Retained,Inactive,No Card,36–45,Zero Balance
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,Retained,Active,Has Card,36–45,High Balance


In [21]:
df.to_csv("../data/processed/bank_customer_churn_cleaned.csv", index=False)
